In [1]:

import numpy as np
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import warnings

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors, metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
from sklearn.utils import resample



In [3]:
train = pd.read_csv(r"\UAIS\datathon\CAW\train.csv")
test = pd.read_csv(r"\UAIS\datathon\CAW\test.csv")
warnings.simplefilter(action='ignore', category=FutureWarning)


In [4]:
train.head()

,fire_id,fire_year,fire_number,fire_name,size_class,fire_location_latitude,fire_location_longitude,fire_origin,general_cause_desc,industry_identifier_desc,...,fuel_type,initial_action_by,ia_arrival_at_fire_date,ia_access,fire_fighting_start_date,fire_fighting_start_size,bucketing_on_fire,distance_from_water_source,first_bucket_drop_date,ex_fs_date
0,0,2015,PWF002,NaN,1,56.108767,-116.840717,Provincial Land,Power Line Industry,NaN,...,O1b,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-04-12 20:12:00
1,1,2011,RWF034,NaN,1,52.397401,-116.072083,Provincial Land,Recreation,NaN,...,NaN,Public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-05-21 10:52:00
2,2,2018,GWF018,NaN,1,56.041000,-119.121650,Private Land,Resident,NaN,...,O1b,Fire Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-09 21:45:00
3,3,2015,HWF081,NaN,2,58.489199,-115.151036,Indian Reservation,Incendiary,NaN,...,O1a,Fire Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-13 19:07:00
4,4,2010,SWF314,NaN,1,55.900885,-115.664957,Indian Reservation,Resident,NaN,...,O1b,Fire Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-08-10 12:00:00


In [5]:
train['size_class'] = train['size_class'] - 1

In [6]:
def replace_values(val):
    stripped_val = str(val).strip()
    if stripped_val == "Surface":
        return "Surface"
    elif stripped_val == "":
        return np.nan
    else:
        return val


train['fire_type'] = train['fire_type'].apply(replace_values)
test['fire_type'] = test['fire_type'].apply(replace_values)

In [7]:
train['Letter_fuel'] = train['fuel_type'].str[0]
test['Letter_fuel'] = test['fuel_type'].str[0]

train['temperature'] = train['temperature'] + 33
test['temperature'] = test['temperature'] + 33


In [8]:
def extract_region(fire_number):

    forest_areas = {
        'C': 'Calgary',
        'E': 'Edson',
        'H': 'High Level',
        'G': 'Grande Prairie',
        'L': 'Lac La Biche',
        'M': 'Fort McMurray',
        'P': 'Peace River',
        'R': 'Rocky',
        'S': 'Slave Lake',
        'W': 'Whitecourt'
    }
    

    region = forest_areas.get(fire_number[0], "Unknown")
    
    return region

train['region'] = train['fire_number'].apply(extract_region)
test['region'] = test['fire_number'].apply(extract_region)


In [9]:
train['has_name'] = train['fire_name'].notnull().astype(int)
test['has_name'] = test['fire_name'].notnull().astype(int)

In [10]:
train['bucketing_on_fire'] = train['bucketing_on_fire'].str.lower()
test['bucketing_on_fire'] = test['bucketing_on_fire'].str.lower()

In [11]:

min_reasonable_date = pd.Timestamp('2000-01-01')
max_reasonable_date = pd.Timestamp('2023-12-31')


date_features = ['fire_start_date', 'discovered_date', 'reported_date', 'dispatch_date','start_for_fire_date', 'assessment_datetime', 'ia_arrival_at_fire_date', 'fire_fighting_start_date', 'first_bucket_drop_date', 'ex_fs_date']

for feature in date_features:

    train[feature] = pd.to_datetime(train[feature], errors='coerce')
    test[feature] = pd.to_datetime(test[feature], errors='coerce')


    train.loc[(train[feature] < min_reasonable_date) | (train[feature] > max_reasonable_date), feature] = pd.NaT
    test.loc[(test[feature] < min_reasonable_date) | (test[feature] > max_reasonable_date), feature] = pd.NaT


In [12]:
weather_mapping = {
    'Clear': 0,
    'CB dry': 1,
    'Cloudy': 2,
    'CB wet': 3,
    'Rain showers': 4
}

train['weather_conditions_encoded'] = train['weather_conditions_over_fire'].map(weather_mapping)
test['weather_conditions_encoded'] = test['weather_conditions_over_fire'].map(weather_mapping)

In [13]:
drop_list = ["fire_name", "fire_number", "discovered_size", "industry_identifier_desc", "weather_conditions_over_fire"]

train = train.drop(columns=drop_list)
test = test.drop(columns=drop_list)


In [14]:
def generate_time_features(df): 

    def timedelta_to_minutes(td):
        return td.total_seconds() / 60 if pd.notna(td) else np.nan


    def compute_time_difference(start, end):
        if pd.isna(start) or pd.isna(end):
            return np.nan
        return timedelta_to_minutes(end - start)

    df['time_to_discovery'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['discovered_date']), axis=1)
    df['time_to_report'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['reported_date']), axis=1)
    df['time_to_dispatch'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['dispatch_date']), axis=1)
    df['time_to_start_for_fire'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['start_for_fire_date']), axis=1)
    df['time_to_assessment'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['assessment_datetime']), axis=1)
    df['time_to_ia_arrival'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['ia_arrival_at_fire_date']), axis=1)
    df['time_to_start_fighting'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['fire_fighting_start_date']), axis=1)
    df['time_to_first_bucket_drop'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['first_bucket_drop_date']), axis=1)
    df['total_time_to_extinguish'] = df.apply(lambda row: compute_time_difference(row['fire_start_date'], row['ex_fs_date']), axis=1)
    
    return df

train = generate_time_features(train)
test = generate_time_features(test)

In [15]:
drop_datelist = ['fire_start_date', 'discovered_date', 'reported_date', 'dispatch_date','start_for_fire_date', 'assessment_datetime', 'ia_arrival_at_fire_date', 'fire_fighting_start_date', 'first_bucket_drop_date', 'ex_fs_date']

train = train.drop(columns=drop_datelist)
test = test.drop(columns=drop_datelist)

In [16]:
train_feature_list = list(train.columns)
test_feature_list = list(test.columns)
label_size_class = LabelEncoder()

for feature in train_feature_list:
 
    if train[feature].dtype == 'object':
        not_null_mask = train[feature].notnull() 
        train.loc[not_null_mask, feature] = label_size_class.fit_transform(train.loc[not_null_mask, feature])

for feature in test_feature_list:

    if test[feature].dtype == 'object':
        not_null_mask = test[feature].notnull()
        test.loc[not_null_mask, feature] = label_size_class.fit_transform(test.loc[not_null_mask, feature])

In [17]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)

In [18]:

def get_gmm_class_feature(feat, n):
    gmm = GaussianMixture(n_components=n, random_state=42)
    gmm.fit(train[feat].values.reshape(-1, 1))
    train[f'{feat}_class'] = gmm.predict(train[feat].values.reshape(-1, 1))
    test[f'{feat}_class'] = gmm.predict(test[feat].values.reshape(-1, 1))

get_gmm_class_feature("fire_location_latitude", 2)
get_gmm_class_feature("fire_location_longitude", 2)
get_gmm_class_feature("assessment_hectares", 6)
get_gmm_class_feature("fire_spread_rate", 8)
get_gmm_class_feature("temperature", 10)
get_gmm_class_feature("relative_humidity", 15)
get_gmm_class_feature("wind_speed", 5)
get_gmm_class_feature("fire_fighting_start_size", 19)
get_gmm_class_feature("distance_from_water_source", 19)
get_gmm_class_feature("time_to_discovery", 6)
get_gmm_class_feature("time_to_report", 19)
get_gmm_class_feature("time_to_dispatch", 7)
get_gmm_class_feature("time_to_start_for_fire", 13)
get_gmm_class_feature("time_to_assessment", 2)
get_gmm_class_feature("time_to_ia_arrival", 7)
get_gmm_class_feature("time_to_start_fighting", 6)
get_gmm_class_feature("time_to_first_bucket_drop", 10)
get_gmm_class_feature("total_time_to_extinguish", 17)


C:\Users\gjftn\AppData\Roaming\Python\Python310\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\gjftn\AppData\Roaming\Python\Python310\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



def get_gmm_class_feature(feat, n):
    gmm = GaussianMixture(n_components=n, random_state=42)
    gmm.fit(train[feat].values.reshape(-1, 1))
    train[f'{feat}_class'] = gmm.predict(train[feat].values.reshape(-1, 1))
    test[f'{feat}_class'] = gmm.predict(test[feat].values.reshape(-1, 1))

def evaluate_classifier():
    model1 = CatBoostClassifier(verbose=0)
    model1.fit(train.drop(columns='size_class'), train['size_class'])
    y_pred1 = model1.predict(test.drop(columns='size_class'))
    return balanced_accuracy_score(test['size_class'], y_pred1)

features_to_optimize = [
    "fire_location_latitude",
    "fire_location_longitude",
    "assessment_hectares",
    "fire_spread_rate",
    "temperature",
    "relative_humidity",
    "wind_speed",
    "fire_fighting_start_size",
    "distance_from_water_source",
    "time_to_discovery",
    "time_to_report",
    "time_to_dispatch",
    "time_to_start_for_fire",
    "time_to_assessment",
    "time_to_ia_arrival",
    "time_to_start_fighting",
    "time_to_first_bucket_drop",
    "total_time_to_extinguish"
]

optimal_components = {}

for feat in features_to_optimize:
    best_score = -np.inf
    best_components = None
    
    for n in range(2, 21):
        get_gmm_class_feature(feat, n)
        score = evaluate_classifier()
        
        if score > best_score:
            best_score = score
            best_components = n
    
    optimal_components[feat] = best_components
    print(f"Optimal components for {feat}: {best_components}, Balanced Accuracy: {best_score:.4f}")

print("All optimal components:", optimal_components)


In [19]:
cols_to_drop = ["fire_id"]

X = train.drop(cols_to_drop + ["size_class"], axis=1)
y = train["size_class"]

X_train_original, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = pd.concat([X_train_original, y_train], axis=1)

max_size = train_data["size_class"].value_counts().max()

lst = [train_data]
for class_index, group in train_data.groupby("size_class"):
    if len(group) < max_size:
        lst.append(resample(group, replace=True, n_samples=max_size-len(group)))

train_data_oversampled = pd.concat(lst)
train_data_oversampled = train_data_oversampled.sample(frac=1).reset_index(drop=True)

X_train_oversampled = train_data_oversampled.drop("size_class", axis=1)
y_train = train_data_oversampled["size_class"]

sc = StandardScaler()
X_train = sc.fit_transform(X_train_oversampled)
X_test = sc.transform(X_test)


In [20]:
def optimize_model(model_eval, pbounds, init_points=5, n_iter=10):
    optimizer = BayesianOptimization(f=model_eval, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=init_points, n_iter=n_iter)
    best_params = optimizer.max['params']
    for key, val in best_params.items():
        if isinstance(val, float) and val.is_integer():
            best_params[key] = int(val)
    return best_params


In [21]:
def catboost_eval(learning_rate, depth, l2_leaf_reg):
    params = {
        'learning_rate': learning_rate,
        'depth': int(depth),
        'l2_leaf_reg': l2_leaf_reg,
        'verbose': 0
    }
    clf = CatBoostClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = balanced_accuracy_score(y_test, y_pred)
    return score

bounds_catboost = {
    'learning_rate': (0.01, 0.3),
    'depth': (4, 8),
    'l2_leaf_reg': (1, 5),
}
best_params_catboost = optimize_model(catboost_eval, bounds_catboost)
best_params_catboost['depth'] = int(best_params_catboost['depth'])
model1 = CatBoostClassifier(**best_params_catboost)
model1.fit(X_train, y_train)


|   iter    |  target   |   depth   | l2_lea... | learni... |
-------------------------------------------------------------
| 1         | 0.7029    | 5.498     | 4.803     | 0.2223    |
| 2         | 0.705     | 6.395     | 1.624     | 0.05524   |
| 3         | 0.6875    | 4.232     | 4.465     | 0.1843    |
| 4         | 0.6961    | 6.832     | 1.082     | 0.2913    |
| 5         | 0.71      | 7.33      | 1.849     | 0.06273   |
| 6         | 0.6888    | 7.885     | 3.53      | 0.2959    |
| 7         | 0.7083    | 5.491     | 4.806     | 0.2547    |
| 8         | 0.7015    | 7.263     | 1.847     | 0.07184   |
| 9         | 0.7054    | 7.772     | 2.472     | 0.07418   |
| 10        | 0.693     | 7.38      | 1.806     | 0.04543   |
| 11        | 0.7062    | 7.812     | 2.462     | 0.05508   |
| 12        | 0.6955    | 5.466     | 4.836     | 0.2644    |
| 13        | 0.6999    | 5.612     | 4.363     | 0.2297    |
| 14        | 0.6766    | 6.441     | 1.886     | 0.2931    |
| 15    

In [22]:

def xgb_eval(learning_rate, max_depth, gamma, colsample_bytree):
    params = {
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'gamma': gamma,
        'colsample_bytree': colsample_bytree
    }
    clf = XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = balanced_accuracy_score(y_test, y_pred)
    return score

bounds_xgboost = {
    'learning_rate': (0.01, 0.3),
    'max_depth': (3, 10),
    'gamma': (0, 1),
    'colsample_bytree': (0.5, 1)
}
best_params_xgboost = optimize_model(xgb_eval, bounds_xgboost)
best_params_xgboost['max_depth'] = int(best_params_xgboost['max_depth'])

model2 = XGBClassifier(**best_params_xgboost)
model2.fit(X_train, y_train)


|   iter    |  target   | colsam... |   gamma   | learni... | max_depth |
-------------------------------------------------------------------------
| 1         | 0.7209    | 0.6873    | 0.9507    | 0.2223    | 7.191     |
| 2         | 0.7191    | 0.578     | 0.156     | 0.02684   | 9.063     |
| 3         | 0.722     | 0.8006    | 0.7081    | 0.01597   | 9.789     |
| 4         | 0.7109    | 0.9162    | 0.2123    | 0.06273   | 4.284     |
| 5         | 0.718     | 0.6521    | 0.5248    | 0.1353    | 5.039     |
| 6         | 0.7056    | 0.937     | 0.9485    | 0.03847   | 9.345     |
| 7         | 0.6841    | 0.5246    | 0.01848   | 0.1841    | 8.067     |
| 8         | 0.7241    | 0.5022    | 0.04037   | 0.1453    | 5.465     |
| 9         | 0.706     | 0.9854    | 0.7096    | 0.2982    | 5.617     |
| 10        | 0.7137    | 0.7358    | 0.1605    | 0.2426    | 3.676     |
| 11        | 0.6994    | 0.9692    | 0.7063    | 0.1405    | 9.989     |
| 12        | 0.7303    | 0.8876    | 

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8875922019901328, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.8821289169846898,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07587868805684211,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [23]:
def rf_eval(n_estimators, max_depth, min_samples_split):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'min_samples_split': int(min_samples_split),
    }
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = balanced_accuracy_score(y_test, y_pred)
    return score

bounds_rf = {
    'n_estimators': (10, 200),
    'max_depth': (5, 40),
    'min_samples_split': (2, 20)
}

best_params_rf = optimize_model(rf_eval, bounds_rf)
best_params_rf['n_estimators'] = int(best_params_rf['n_estimators'])
best_params_rf['max_depth'] = int(best_params_rf['max_depth'])
best_params_rf['min_samples_split'] = int(best_params_rf['min_samples_split'])

model3 = RandomForestClassifier(**best_params_rf)
model3.fit(X_train, y_train)


|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.6825    | 18.11     | 19.11     | 149.1     |
| 2         | 0.6398    | 25.95     | 4.808     | 39.64     |
| 3         | 0.7127    | 7.033     | 17.59     | 124.2     |
| 4         | 0.6269    | 29.78     | 2.371     | 194.3     |
| 5         | 0.6568    | 34.14     | 5.822     | 44.55     |
| 6         | 0.6638    | 26.15     | 20.0      | 112.3     |
| 7         | 0.7017    | 6.7       | 16.72     | 124.3     |
| 8         | 0.7116    | 9.482     | 19.06     | 122.2     |
| 9         | 0.705     | 10.98     | 19.89     | 126.4     |
| 10        | 0.6983    | 6.784     | 19.36     | 132.2     |
| 11        | 0.6746    | 5.195     | 18.58     | 119.7     |
| 12        | 0.7058    | 5.128     | 19.6      | 127.5     |
| 13        | 0.6874    | 5.0       | 20.0      | 141.6     |
| 14        | 0.6796    | 14.22     | 14.33     | 136.1     |
| 15    

RandomForestClassifier(max_depth=7, min_samples_split=17, n_estimators=124)

In [24]:

def lgbm_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    params = {
        'num_leaves': int(num_leaves),
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_split_gain': min_split_gain,
        'min_child_weight': min_child_weight,
    }
    clf = LGBMClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = balanced_accuracy_score(y_test, y_pred)
    return score

bounds_lgbm = {
    'num_leaves': (31, 500),
    'feature_fraction': (0.1, 0.9),
    'bagging_fraction': (0.8, 1),
    'max_depth': (5, 8.99),
    'lambda_l1': (0, 5),
    'lambda_l2': (0, 3),
    'min_split_gain': (0.001, 0.1),
    'min_child_weight': (5, 50),
}
best_params_lgbm = optimize_model(lgbm_eval, bounds_lgbm)
best_params_lgbm['max_depth'] = int(best_params_lgbm['max_depth'])
best_params_lgbm['num_leaves'] = int(best_params_lgbm['num_leaves'])

model4 = LGBMClassifier(**best_params_lgbm)
model4.fit(X_train, y_train)


|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] feature_fraction is set=0.8605714451279329, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8605714451279329
[LightGBM] [Warning] lambda_l1 is set=3.6599697090570253, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.6599697090570253
[LightGBM] [Warning] lambda_l2 is set=1.7959754525911098, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7959754525911098
[LightGBM] [Warning] bagging_fraction is set=0.8749080237694725, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8749080237694725
[LightGBM] [Warning] feature_fraction is set=0.8605714451279329, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8605714451279329
[LightGBM] [Warning] lambda_l1 is 

LGBMClassifier(bagging_fraction=0.9273691890210161,
               feature_fraction=0.45750404170141123,
               lambda_l1=3.3373122678939495, lambda_l2=2.1105719535406227,
               max_depth=8, min_child_weight=33.82274159804797,
               min_split_gain=0.07421543482337388, num_leaves=254)

In [25]:
y_pred1 = model1.predict(X_test)
print("CatBoost Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred1))

y_pred2 = model2.predict(X_test)
print("XGBoost Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred2))

y_pred3 = model3.predict(X_test)
print("RandomForest Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred3))


y_pred4 = model4.predict(X_test)
print("LightGBM Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred4))

weights = [0.25, 0.25, 0.0, 0.5] 

probs1 = model1.predict_proba(X_test)
probs2 = model2.predict_proba(X_test)
probs3 = model3.predict_proba(X_test)
probs4 = model4.predict_proba(X_test)
    
final_probs = (weights[0] * probs1 + weights[1] * probs2 + 
               weights[2] * probs3 + weights[3] * probs4)

final_probs[:, 3] += 0.0 * final_probs[:, 2]
final_probs[:, 2] -= 0.0 * final_probs[:, 2]


final_probs /= final_probs.sum(axis=1, keepdims=True)


final_pred = np.argmax(final_probs, axis=1)

ensemble_balanced_accuracy = balanced_accuracy_score(y_test, final_pred)
print("Ensemble Balanced Accuracy with Weighted Soft Voting after adjustment:", ensemble_balanced_accuracy)

CatBoost Balanced Accuracy: 0.7099997723027838
XGBoost Balanced Accuracy: 0.7303264609283825
RandomForest Balanced Accuracy: 0.7098959875675778
[LightGBM] [Warning] feature_fraction is set=0.45750404170141123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45750404170141123
[LightGBM] [Warning] lambda_l1 is set=3.3373122678939495, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.3373122678939495
[LightGBM] [Warning] lambda_l2 is set=2.1105719535406227, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.1105719535406227
[LightGBM] [Warning] bagging_fraction is set=0.9273691890210161, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9273691890210161
LightGBM Balanced Accuracy: 0.7290264525268949
[LightGBM] [Warning] feature_fraction is set=0.45750404170141123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45750404170141123
[LightGBM] [Warning] lambda_l1 is set=3.3373122678939495, reg_alpha=0.0 will be ign

In [26]:
#y_test_1d = np.ravel(y_test)
#final_pred_1d = np.ravel(final_pred) 

#mismatches = y_test_1d != final_pred_1d

#mismatched_probs = final_probs[mismatches]

#mismatched_data = pd.DataFrame({
#    'fire_id': test.loc[mismatches, 'fire_id'],
#    'Actual_size_class': y_test_1d[mismatches],
#    'Predicted': final_pred_1d[mismatches],
#    'Prob_Class_0': mismatched_probs[:, 0],
#    'Prob_Class_1': mismatched_probs[:, 1],
#    'Prob_Class_2': mismatched_probs[:, 2],
#    'Prob_Class_3': mismatched_probs[:, 3],
#    'Prob_Class_4': mismatched_probs[:, 4]
#})


#mismatched_data.to_csv('mismatched_data_with_probs.csv', index=False)


In [27]:
print(classification_report(y_test, final_pred))
print(confusion_matrix(y_test, final_pred))
ensemble_accuracy = accuracy_score(y_test, final_pred)
print(ensemble_accuracy)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2066
           1       0.88      0.88      0.88       930
           2       0.60      0.73      0.66       162
           3       0.57      0.38      0.46        65
           4       0.63      0.64      0.63        50

    accuracy                           0.91      3273
   macro avg       0.73      0.72      0.72      3273
weighted avg       0.91      0.91      0.91      3273

[[1994   72    0    0    0]
 [  57  819   53    1    0]
 [   1   28  119   10    4]
 [   1    6   18   25   15]
 [   1    1    8    8   32]]
0.9132294531011305


In [28]:
probabilities = model1.predict_proba(X_test)
print(probabilities)
df_probabilities = pd.DataFrame(probabilities)
df_probabilities.to_csv('probabilities.csv', index=False)

[[9.61419125e-03 9.87376873e-01 2.98490826e-03 1.37553625e-05
  1.02721678e-05]
 [2.59260549e-03 9.03499211e-01 9.29726442e-02 6.58115695e-04
  2.77423361e-04]
 [9.93924072e-01 6.04320681e-03 3.11331647e-05 1.32990222e-06
  2.57859877e-07]
 ...
 [1.39855270e-02 9.74440958e-01 1.13433554e-02 8.81137582e-05
  1.42045770e-04]
 [6.45990224e-04 1.49731826e-01 7.46866086e-01 8.59442886e-02
  1.68118096e-02]
 [9.85041805e-01 1.49522099e-02 3.56317157e-06 1.80933527e-06
  6.12223344e-07]]


In [29]:
X_test_real = test.drop(cols_to_drop, axis=1)
X_test_real_transformed = sc.transform(X_test_real)

y_pred1_real = model1.predict(X_test_real_transformed)
y_pred2_real = model2.predict(X_test_real_transformed)
y_pred3_real = model3.predict(X_test_real_transformed)
y_pred4_real = model4.predict(X_test_real_transformed)

weights = [0.25, 0.25, 0.0, 0.5] 

probs1_real = model1.predict_proba(X_test_real_transformed)
probs2_real = model2.predict_proba(X_test_real_transformed)
probs3_real = model3.predict_proba(X_test_real_transformed)
probs4_real = model4.predict_proba(X_test_real_transformed)
    
final_probs_real = (weights[0] * probs1_real + weights[1] * probs2_real + 
               weights[2] * probs3_real + weights[3] * probs4_real)

final_probs_real[:, 3] += 0.0 * final_probs_real[:, 2]
final_probs_real[:, 2] -= 0.0 * final_probs_real[:, 2]


final_probs_real /= final_probs_real.sum(axis=1, keepdims=True)


final_pred_real = np.argmax(final_probs_real, axis=1)

[LightGBM] [Warning] feature_fraction is set=0.45750404170141123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45750404170141123
[LightGBM] [Warning] lambda_l1 is set=3.3373122678939495, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.3373122678939495
[LightGBM] [Warning] lambda_l2 is set=2.1105719535406227, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.1105719535406227
[LightGBM] [Warning] bagging_fraction is set=0.9273691890210161, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9273691890210161
[LightGBM] [Warning] feature_fraction is set=0.45750404170141123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45750404170141123
[LightGBM] [Warning] lambda_l1 is set=3.3373122678939495, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.3373122678939495
[LightGBM] [Warning] lambda_l2 is set=2.1105719535406227, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.1105719535406227
[LightG

In [30]:
#rfc.feature_importances_

In [31]:
my_submission = pd.DataFrame({'fire_id': test.index, 'size_class': final_pred_real+1})
my_submission.to_csv('submission.csv', index=False)